In [ ]:
from IPython.display import HTML

import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
%matplotlib inline

# ProfileTK
#### (a profiling toolkit)

This package combines pyinstrument, line_profiler, memory_profiler, tracemalloc, and pycallgraph
in order to provide a seemless and constant interface between them.  We provide timing profiles
that can be analyized across different parameters, standalone timing profiles by function call
and line by line, memory profiles by function call and line by line, and call graphs color coded 
by runtime costs.

In [ ]:
import profiletk

In [ ]:
ptk = profiletk.ProfileTK()




### Let's walk through an example where we're interested in gathering timing information across many different parameters.  In this case we're interested in looking at comparing the time it takes to open different types of netcdf files and using different arguments.

First we'll set the files we'll loop over.

In [ ]:
fns = [
    'wrfout_d01_1979-01-01-01_00:00:00_2D.nc',
    'wrfout_d01_1979-01-01-01_00:00:00_3D.nc',
    'mpas_output.nc',
    'b40.20th.track1.1deg.006.cam2.h0.TS.185001-185012.nc',
    'b40.20th.track1.1deg.006.cam2.h0.T.185001-185012.nc',
    'b.e21.BWmaHIST.f19_g17.PMIP4-past1000.001.cice.h.0850-01.nc',
    'b.e21.BWmaHIST.f19_g17.PMIP4-past1000.001.clm2.h0.0850-01.nc',
    'b.e21.BWmaHIST.f19_g17.PMIP4-past1000.001.pop.h.0850-01.nc',
    'b.e21.BWmaHIST.f19_g17.PMIP4-past1000.001.cam.h0.0850-01.nc'
]

Now we'll loop over those file names with two different sets of parameters and get a set of timings.  To do this, we'll call `ptk.collect_functional_times`.
```
    def collect_functional_times(self, key: str, func: callable, *args, **kwargs):
        
        Profiles a function with pyinstrument and populate the new timings into the dataframe.

        Parameters
        ----------
        key : str
            The unique identifier asigned to this new entry within the dataframe.
        func : callable
            The function to profile. 
        args : list, optional
            A list of arguments to pass to the function that is being profiled.
        kwargs : dict, optional
            A dictionary of keyword, value arguments to pass to the function that is being profiled.

        
```

In [ ]:
for fn in fns:
    key = fn+'.defaults'
    ptk.collect_functional_times(key, xr.open_dataset, 'data/'+fn)
for fn in fns:
    key = fn+'.preferred'
    ptk.collect_functional_times(key, xr.open_dataset, 'data/'+fn, decode_times=False, mask_and_scale=False, decode_cf=False)
    

Now we can try printing the dataframe we just filled.

In [ ]:
ptk.timingsDF

We can use query the dataframe for the different keys that were added.

In [ ]:
pd.options.display.max_colwidth = 10000
pd.set_option('display.width', 1000)
ptk.timingsDF['key']

We can use the dataframe for any type of analysis we're interested in.  Here we create a line plot that looks at the time it takes to run the selected functions across the different files.

In [ ]:
ptk.timingsDF.sort_values('open_dataset').plot(x="key", y=['open_dataset', 'decode_cf_variable', 'as_variable'], alpha=0.50, rot=90)

Here we create a horizontal bar plot that plots the time to run a function across all keys.

In [ ]:
import seaborn as sns

sns.factorplot(y='key',x='as_variable',kind='bar',data=ptk.timingsDF.sort_values('as_variable'))

We can also use the `ptk.profiles` object to get the full timeline profile from pyinstrument.

In [ ]:
print(ptk.profiles['mpas_output.nc.preferred'])

Here we call the `ptk.collect_functional_times` function for a function that we created.

In [ ]:
def open_read(fn):
    ds = xr.open_dataset(fn)
    for v in ds.variables:
        t = v
for fn in fns:
    key = fn+'.defaults'
    ptk.collect_functional_times(key=key, func=open_read, fn='data/'+fn)

## Other functionality we provide

#### Create a call graph.

    def show_call_graph(self, func: callable, fn: str, *args, **kwargs):
        """
        Creates a call graph for the speficed function that is color coded to indicate which functions
        take longer to run.

        This requires an install of graphviz (https://www.graphviz.org/).  

        Parameters
        ----------
        func : callable
            The function to profile. 
        fn : str
            The name of the filename to save the graph image to. 
        args : list, optional
            A list of arguments to pass to the function that is being profiled.
        kwargs : dict, optional
            A dictionary of keyword, value arguments to pass to the function that is being profiled.



In [ ]:
from IPython.display import Image

img_fn = 'call_graph.png'
ptk.show_call_graph(func=xr.open_dataset,
                    fn=img_fn,filename_or_obj='data/wrfout_d01_1979-01-01-01_00:00:00_2D.nc', 
                    decode_times=False, mask_and_scale=False, decode_cf=False)

Image(filename = img_fn)

#### We also provide a function that prints the hotspots of a function call.

    def print_timer_hotspots(self, key: str, l=25):
        
        Prints the function names within the specified profile in descenting oder based on run time.
        This function prints the rank, the run time, the function name, file name and line number, and 
        the call tree depth.  This function uses the timing information provided by pyinstrument gathered 
        from the call to collect_functional_times. 

        Parameters
        ----------
        key : str
            The unique identifier of a profile that was set within the call to collect_functional_times.
        l : int
            The number of functions to print.
 
        


In [ ]:
ptk.print_timer_hotspots('mpas_output.nc.defaults', l=25)

#### A print out of the line by line timings for the selected functions.

    def collect_linebyline_times(self, func: callable, o_funcs: list, *args, **kwargs):
        
        Calls the line_profiler utility to collect line by line timing information for specified functions.  
        New information is gathered each time this function is called so some variability exists between calls. 

        Parameters
        ----------
        func : callable
            The function to profile. 
        o_funcs : list
            A list of additional functions to profile.  These must be in the call tree of the function
            specified in the func argument.
        args : list, optional
            A list of arguments to pass to the function that is being profiled.
        kwargs : dict, optional
            A dictionary of keyword, value arguments to pass to the function that is being profiled.

        

In [ ]:
ptk.collect_linebyline_times(func=xr.open_dataset, 
                             o_funcs=[xr.backends.api._protect_dataset_variables_inplace],
                             filename_or_obj='data/wrfout_d01_1979-01-01-01_00:00:00_2D.nc', 
                             decode_times=False, mask_and_scale=False, decode_cf=False)

#### A print out of the line by line memory usage for the selected functions.

    def collect_linebyline_memory_usage(self, func: callable, o_funcs: list, *args, **kwargs):
        
        Calls the memory_profiler utility to collect line by line memory information for specified functions.  
        New information is gathered each time this function is called so some variability exists between calls. 

        Parameters
        ----------
        func : callable
            The function to profile. 
        o_funcs : list
            A list of additional functions to profile.  These must be in the call tree of the function
            specified in the func argument.
        args : list, optional
            A list of arguments to pass to the function that is being profiled.
        kwargs : dict, optional
            A dictionary of keyword, value arguments to pass to the function that is being profiled.

        


In [ ]:
ptk.collect_linebyline_memory_usage(func=xr.open_dataset, 
                                    o_funcs=[xr.conventions.decode_cf_variable, xr.core.variable.as_variable],
                                    filename_or_obj='data/wrfout_d01_1979-01-01-01_00:00:00_2D.nc', 
                                    decode_times=False, mask_and_scale=False, decode_cf=False)

#### Print out the highwater mark for memory usage for the selected function.

    def collect_memory_usage(self, func: callable, *args, **kwargs):
        
        Calls the memory_profiler utility to collect the high water memory information for the specified function.  
        New information is gathered each time this function is called so some variability exists between calls. 

        Parameters
        ----------
        func : callable
            The function to profile. 
        args : list, optional
            A list of arguments to pass to the function that is being profiled.
        kwargs : dict, optional
            A dictionary of keyword, value arguments to pass to the function that is being profiled.

 


In [ ]:
ptk.collect_memory_usage(func=xr.open_dataset,
                         filename_or_obj='data/wrfout_d01_1979-01-01-01_00:00:00_2D.nc', 
                         decode_times=False, mask_and_scale=False, decode_cf=False)

#### Print the memory hotspots.

    def print_memory_hotspots(self, func: callable, l=10, *args, **kwargs):
        
        Prints the memory hotspots within the selected function by calling tracemalloc in descending order.  
        This function prints the rank, the location of the code, size, and line of code.  New information
        is gathered each time this function is called so some variability may exist between calls.  

        Parameters
        ----------
        func : callable
            The function to profile. 
        l : int
            The number of hotspots to print.
        args : list, optional
            A list of arguments to pass to the function that is being profiled.
        kwargs : dict, optional
            A dictionary of keyword, value arguments to pass to the function that is being profiled.         



In [ ]:
ptk.print_memory_hotspots(func=xr.open_dataset,
                          l=5,
                          filename_or_obj='data/wrfout_d01_1979-01-01-01_00:00:00_2D.nc', 
                          decode_times=False, mask_and_scale=False, decode_cf=False)